This finds all the job codes of a search of Data - Kent - radius=10 on Reed.co.uk

In [140]:
x = np.arange(2,63+1)
a = "1"
b = 2
#x = #np.append(x,[b],axis = 0)
print(x)

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


In [121]:
### My reed api_key: 96a7ec49-549c-4529-b9d2-fa3059a437b3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


#pages = np.range(63)

#url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=63&proximity=30"
url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"

def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def find_jobs_iterate(url):
  site_html = get_html(url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  jobs_found = results.find_all(class_="job-result-card")

  for job in jobs_found:
      job_id = int(job["id"].split("jobSection")[1])
      
      np.append(id_list,[job_id])
  return id_list
  
def find_total_pages(site_html):
  of_total_jobs = site_html.find(class_ = "search-results-footer").text
  #print(of_total_jobs)
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]
  total_jobs = total_jobs_split1.split("of ")[1]
  print(total_jobs)
#not used

#soup = get_html(url)
#id_list = find_jobs(soup)

#print(len(id_list))

['48421098', '48327271', '48065396', '48329695', '48414598', '48400496', '48297265', '48203555', '48472751', '48022309', '47821724', '48120315', '48118027', '48116135', '48106997', '48105197', '48034421', '48460519', '48460518', '48199033', '48089189', '48020944', '48413631', '48323820']
1,574
24


In [ ]:
initial_url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"


def first_scrape(url):
  soup = get_html(url)
  results = soup.find(class_="col-sm-8 col-md-9 results-container")         #
  jobs_found = results.find_all(class_="job-result-card")                   #
  of_total_jobs = results.find(class_ = "search-results-footer").text       #
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]                       #
  total_jobs = total_jobs_split1.split("of ")[1]                            #

  total_pages = int(np.ceil(total_jobs/25))
  total_id_list = np.array([])

  for page_no in np.arange(2,3+1):
    url2 = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno="+page_no+"&proximity=30"
    page_id_list = find_jobs_iterate(url2)

    np.append(total_id_list,[page_id_list])






In [114]:
def call_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  url = ""
  job_df = pd.DataFrame()


  for job_id in id_list:
    url = base_url+job_id
    post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) 
    json_data = post_request.json()


    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df
job_df = call_api(id_list)
display(job_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   employerId           25 non-null     int64 
 1   employerName         25 non-null     object
 2   jobId                25 non-null     int64 
 3   jobTitle             25 non-null     object
 4   locationName         25 non-null     object
 5   minimumSalary        16 non-null     object
 6   maximumSalary        16 non-null     object
 7   yearlyMinimumSalary  16 non-null     object
 8   yearlyMaximumSalary  16 non-null     object
 9   currency             16 non-null     object
 10  salaryType           25 non-null     object
 11  salary               16 non-null     object
 12  datePosted           25 non-null     object
 13  expirationDate       25 non-null     object
 14  externalUrl          12 non-null     object
 15  jobUrl               25 non-null     object
 16  partTime  

None

In [115]:
print(job_df.columns)

job_df.drop(columns = ['employerId','minimumSalary', 'maximumSalary', 'yearlyMinimumSalary',
       'yearlyMaximumSalary', 'currency', 'salaryType', 'salary', 'datePosted',
       'expirationDate','partTime', 'fullTime', 'contractType', 'jobDescription', 'applicationCount'],inplace=True)

display(job_df)

Index(['employerId', 'employerName', 'jobId', 'jobTitle', 'locationName',
       'minimumSalary', 'maximumSalary', 'yearlyMinimumSalary',
       'yearlyMaximumSalary', 'currency', 'salaryType', 'salary', 'datePosted',
       'expirationDate', 'externalUrl', 'jobUrl', 'partTime', 'fullTime',
       'contractType', 'jobDescription', 'applicationCount'],
      dtype='object')


,employerName,jobId,jobTitle,locationName,externalUrl,jobUrl
0,Reed,48401296,Data Lead,Folkestone,None,https://www.reed.co.uk/jobs/data-lead/48401296
1,Senitor Associates,48120219,Data Analyst,Sittingbourne,None,https://www.reed.co.uk/jobs/data-analyst/48120219
2,Morgan Sindall Property Services,48223150,Data Analyst,Billericay,https://morgansindall.octo-firstclass.co.uk/ca...,https://www.reed.co.uk/jobs/data-analyst/48223150
3,IronmongeryDirect & ElectricalDirect,48320718,Data Analyst,Basildon,None,https://www.reed.co.uk/jobs/data-analyst/48320718
4,OnetoOne Personnel,48465441,Data Analyst,Leigh-on-Sea,https://jobs.onetoonepersonnel.co.uk/job/data-...,https://www.reed.co.uk/jobs/data-analyst/48465441
5,Reed,48401295,Data Lead,Folkestone,None,https://www.reed.co.uk/jobs/data-lead/48401295
6,Morgan Sindall Property Services,48211407,Data Analyst,Dartford,https://morgansindall.octo-firstclass.co.uk/ca...,https://www.reed.co.uk/jobs/data-analyst/48211407
7,P3 Search & Selection,48350103,"Data Engineer - Data Lake, Data Pipelines, BI",Sevenoaks,None,https://www.reed.co.uk/jobs/data-engineer-data...
8,Treatwell,48136108,Data Analytics Engineer,United Kingdom,https://apply.workable.com/treatwell/j/BFCF350...,https://www.reed.co.uk/jobs/data-analytics-eng...
9,Huntress,48389352,Graduate Data Analyst,Tenterden,None,https://www.reed.co.uk/jobs/graduate-data-anal...


In [ ]:
#@title Default title text
###Failed
"""
key = "96a7ec49-549c-4529-b9d2-fa3059a437b3"

exm_url = """https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&loc 
ationName={locationName}&employerId={employerId}&distanceFromLocation={distance in miles}"""


url = "https://www.reed.co.uk/api/1.0/search?keywords=data&location=ashfordkent&distanceFromLocation=10" #

post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']

if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)

json_keys = json_data.keys()
print("total results found: ",json_data["totalResults"])
#'results' has the top 100 results from the api
#'ambiguousLocations' is empty
#'totalResults' is: 43984, the total value found by the api
print(json_keys) #.keys()
2
results_df = pd.json_normalize(json_data["results"]) 
#ambi_df = pd.json_normalize(json_data["ambiguousLocations"])
display(results_df.info())

results_df.drop(columns = ["minimumSalary", 'currency', 'expirationDate', 'date','jobDescription',
       'applications',"employerProfileName","employerProfileId","maximumSalary"],inplace=True)


def outside_kent():
  drop_cities = ["Bristol", "Manchester","Colchester","Reading","Brussels","Glasgow","Edinburgh","Southampton","Oxford","Cambridge"]
  for city in drop_cities:
    results_df.drop(results_df[results_df["locationName"] == city].index, inplace =True)
#outside_kent()

#results_df.drop("locationName")
display(results_df)
less_results_df = results_df[['employerName', 'jobTitle' ,'locationName']]
#display(less_results_df)
"""

In [98]:
###api test

job_id = "48445740"
print(job_id)
url = "https://www.reed.co.uk/api/1.0/jobs/" # 48401296

url = url+job_id
post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']
if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)
print(json_data)

df = pd.json_normalize(json_data)
df




48445740
connection made, code:  200
{'employerId': 408731, 'employerName': 'International SOS', 'jobId': 48445740, 'jobTitle': 'Data Analyst', 'locationName': 'London', 'minimumSalary': None, 'maximumSalary': None, 'yearlyMinimumSalary': None, 'yearlyMaximumSalary': None, 'currency': None, 'salaryType': 'per annum', 'salary': None, 'datePosted': '07/10/2022', 'expirationDate': '06/11/2022', 'externalUrl': 'https://international-sos-pte-ltd.contactrh.com/jobs/5114/40712131', 'jobUrl': 'https://www.reed.co.uk/jobs/data-analyst/48445740', 'partTime': False, 'fullTime': True, 'contractType': 'Permanent', 'jobDescription': ' <p>Company description:</p> <p>International SOS is the world’s leading medical and security services company with over 12,000 employees working in 1,000 locations in 90 countries.&nbsp;International SOS Government Services administers the TRICARE Overseas Program providing high-quality, accessible  health care services for U.S. Military service members, family members

,employerId,employerName,jobId,jobTitle,locationName,minimumSalary,maximumSalary,yearlyMinimumSalary,yearlyMaximumSalary,currency,...,salary,datePosted,expirationDate,externalUrl,jobUrl,partTime,fullTime,contractType,jobDescription,applicationCount
0,408731,International SOS,48445740,Data Analyst,London,None,None,None,None,None,...,None,07/10/2022,06/11/2022,https://international-sos-pte-ltd.contactrh.co...,https://www.reed.co.uk/jobs/data-analyst/48445740,False,True,Permanent,<p>Company description:</p> <p>International ...,22
